In [1]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [2]:

from nltk.stem import PorterStemmer

def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Remove punctuation and special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    
    # Tokenize the text
    words = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    
    # Stem words
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    
    # Join the words back into a single string
    preprocessed_text = ' '.join(words)
    
    return preprocessed_text
a1="List is collection of non homogenious data type it is mutable in nature"
a2=""
a3=""
cleaned_audio_text=preprocess_text(a1)


print(cleaned_audio_text)




list collect non homogeni data type mutabl natur


In [3]:
import pandas as pd

ds = pd.read_csv("question_set.csv", encoding='latin1')
ds

,sr_no,question,ans,ans1
0,1.0,What is Python? List some popular applications...,"Python is a widely-used general-purpose, high-...","Python is a widely-used general-purpose, high-..."
1,2.0,What are the benefits of using Python language...,Object-Oriented Language\nHigh-Level Language\...,Object-Oriented Language\nHigh-Level Language\...
2,3.0,What is the difference between a Mutable datat...,"Mutable data types can be edited i.e., they ca...","Mutable data types can be edited i.e., they ca..."
3,4.0,What is the difference between a Set and Dicti...,The set is an unordered collection of data typ...,The set is an unordered collection of data typ...
4,5.0,What is a pass in Python?,Pass means performing no operation or in other...,Pass means performing no operation or in other...
5,6.0,Difference between for loop and while loop in ...,The for Loop is generally used to iterate th...,The for Loop is generally used to iterate th...
6,NaN,NaN,NaN,NaN


In [4]:
ds["ans1"]=ds["ans1"].astype(str).apply(preprocess_text)

In [5]:
new_ds=ds[['sr_no','question','ans','ans1']]
new_ds['ans1']=new_ds['ans1'].apply(lambda x:"".join(x))

In [6]:
import pickle
with open("preprocess_data.pkl",'wb')as file:
    pickle.dump(new_ds,file)

In [7]:
import pickle
new_ds = pickle.load(open('preprocess_data.pkl', 'rb'))
new_ds

,sr_no,question,ans,ans1
0,1.0,What is Python? List some popular applications...,"Python is a widely-used general-purpose, high-...",python widelyus generalpurpos highlevel progra...
1,2.0,What are the benefits of using Python language...,Object-Oriented Language\nHigh-Level Language\...,objectori languag highlevel languag dynam type...
2,3.0,What is the difference between a Mutable datat...,"Mutable data types can be edited i.e., they ca...",mutabl data type edit ie chang runtim eg list ...
3,4.0,What is the difference between a Set and Dicti...,The set is an unordered collection of data typ...,set unord collect data type iter mutabl duplic...
4,5.0,What is a pass in Python?,Pass means performing no operation or in other...,pass mean perform oper word placehold compound...
5,6.0,Difference between for loop and while loop in ...,The for Loop is generally used to iterate th...,loop gener use iter element variou collect typ...
6,NaN,NaN,NaN,nan


In [8]:
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [9]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [10]:
new_ds['ans']=new_ds['ans'].apply(stem)

AttributeError: 'float' object has no attribute 'split'

In [ ]:
new_ds['ans']=new_ds['ans'].apply(lambda x:x.lower())

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Function to get the top 1000 repeated words from a text
def get_top_words(text, n=30):
    # Tokenize the text into words
    words = text.split()
    # Count the frequency of each word
    word_freq = {}
    for word in words:
        word_freq[word] = word_freq.get(word, 0) + 1
    # Sort words by frequency and select the top n
    top_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)[:n]
    return [word[0] for word in top_words]

# Get the top 1000 repeated words from your resume
top_ans_word = get_top_words(cleaned_audio_text)

# Get the top 1000 repeated words from your dataset (assuming 'new_ds' is your dataset)
all_ans = new_ds['ans'].tolist()
top_dataset_words = get_top_words(' '.join(all_ans))

print(top_dataset_words,"\n\n")
print(top_ans_word)

# Combine the top words from resume and dataset
top_words = list(set(top_ans_word + top_dataset_words))

# Create a TF-IDF vectorizer with the top words
tfidf_vectorizer = TfidfVectorizer(vocabulary=top_words)

# Calculate TF-IDF vectors for your resume and each company description
tfidf_matrix = tfidf_vectorizer.fit_transform([cleaned_audio_text] + all_ans)

# Calculate cosine similarities
cosine_similarities = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1:]).flatten()

# Add similarity scores to the DataFrame
new_ds['Similarity Score'] = cosine_similarities*100

# Rank the job descriptions based on similarity scores
new_ds = new_ds.sort_values(by='Similarity Score', ascending=False)

# Filter job descriptions with a similarity score threshold
threshold = 0.1  # Adjust as needed
filtered_df = new_ds[new_ds['Similarity Score'] >= threshold]

# Sort the DataFrame by Similarity Score in descending order
filtered_df = filtered_df.sort_values(by='Similarity Score', ascending=False)

# Select the top 3 companies with the highest similarity scores
top_5_companies = filtered_df.nlargest(1, 'Similarity Score')

# Assign ratings to your resume for the top 3 companies
top_5_companies['Ans_Rating'] = top_5_companies['Similarity Score']  # Example rating scale

# Display the top 3 companies
print(top_5_companies[['ans','Ans_Rating']])


['the', 'is', 'and', 'a', 'in', 'of', 'type', 'data', 'use', 'loop', 'python', 'it', 'to', 'can', 'be', 'they', 'develop', 'an', 'languag', 'collect', 'where', 'high-level', 'program', 'language.', 'wa', 'by', 'programm', 'portabl', 'oper', 'mutabl'] 


['list', 'collect', 'non', 'homogeni', 'data', 'type', 'mutabl', 'natur']
                                                 ans  Ans_Rating
3  the set is an unord collect of data type that ...   34.600356
